In [ ]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False
if IN_COLAB:
    !pip install -q transformers
    from google.colab import drive
    drive.mount('/content/drive')
    import sys
    sys.path.append('/content/drive/MyDrive/gan_bert_for_cpv')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 42.8 MB/s eta 0:00:00
Mounted at /content/drive


In [ ]:
from transformers import logging
logging.set_verbosity_error()
import torch
from create_dataset import CreateDataset
from CPVDataset import CPVDataset
from process_data import ProcessData
from utils import *
from torch.utils.data import DataLoader
from run_model import RunModel
from models import BertGenerator, BertDiscriminator,EmbeddingClassifier
from transformers import AutoTokenizer, AutoModel, AutoConfig

In [ ]:
label_column = "klasse"
text_column = "Korte beschrijving aanbesteding"
input_df = pd.read_csv("/content/drive/MyDrive/gan_bert_for_cpv/data/cleaned_dutch_ted1.csv")
input_df2=pd.read_csv("/content/drive/MyDrive/gan_bert_for_cpv/data/cleaned_foreign_ted1.csv")
all_input=pd.concat([input_df,input_df2])

In [ ]:
vc=all_input[label_column].value_counts()

In [ ]:
running_device = torch.device("cuda")

In [ ]:
prd = ProcessData(input_df, label_column, text_column, sample_num=1000, aug_limit=500,test_per_label_num=30, drop_limit=50,
                  balanced_test=True)
test_df, suf_df, insuf_df, label_encoder, onehot = prd.process()
# balanced_test=test_df.sample(frac=1).groupby(label_column, sort=False).head(30)
value_counts = insuf_df[label_column].value_counts()
model_runner = RunModel(running_device)

After removing duplicate sentence: 60385
After drop cpv with low observations we have : 51918
The num of class is  198
The number of total data is:  44031
Total number of test data is:  5940
Total number of train data is:  38091


In [ ]:
value_counts

Volledige of gedeeltelijke bouw- en civieltechnische werkzaamheden    464
Wegenuitrusting                                                       433
Onderzoek en ontwikkeling, en aanverwante adviezen                    426
Reparatie- en onderhoudsdiensten                                      393
Zware motorvoertuigen                                                 388
                                                                     ... 
Liftonderhoud                                                          22
Lampen en lamparmaturen                                                21
Abonnementsdiensten                                                    21
Diensten voor technische testen, analyse en adviezen                   21
Industriële gassen                                                     21
Name: klasse, Length: 180, dtype: int64

In [ ]:
from joblib import dump, load
dump(onehot, '/content/drive/MyDrive/gan_bert_for_cpv/output/description/onehot.joblib')

['/content/drive/MyDrive/gan_bert_for_cpv/output/description/onehot.joblib']

In [ ]:
dump(label_encoder, '/content/drive/MyDrive/gan_bert_for_cpv/output/description/labelencoder.joblib')

['/content/drive/MyDrive/gan_bert_for_cpv/output/description/labelencoder.joblib']

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("GroNLP/bert-base-dutch-cased")
config = AutoConfig.from_pretrained("GroNLP/bert-base-dutch-cased", output_hidden_states=True)
bert = AutoModel.from_pretrained("GroNLP/bert-base-dutch-cased", config=config).to(running_device)

In [ ]:
suf_dataset=CPVDataset(suf_df, label_column, text_column, running_device, bert, tokenizer, 0, cls_rep=False, max_len=100, label_encoder=label_encoder,onehot_encoder=onehot)
test_dataset=CPVDataset(test_df, label_column, text_column, running_device, bert, tokenizer, 0, cls_rep=False, max_len=100, label_encoder=label_encoder,onehot_encoder=onehot)

In [ ]:
test_dl = DataLoader(test_dataset, shuffle=True, batch_size=20)
train_dl = DataLoader(suf_dataset, shuffle=True, batch_size=20)

In [ ]:
insuf_ds_dict = {}
for label in set(insuf_df[label_column].values):
    sub_df = insuf_df[insuf_df[label_column] == label]
    sub_ds = CPVDataset(sub_df, label_column, text_column, running_device, bert, tokenizer, 0, cls_rep=False, max_len=100, label_encoder=label_encoder,onehot_encoder=onehot)
    insuf_ds_dict[label] = sub_ds

In [ ]:
aug_limits = decide_aug_labels(True, 800, value_counts,1)
augmented_ds_lists = model_runner.augment_data(bert, aug_limits, insuf_ds_dict,batch_size=20, g_epochs=10, glr=2e-6, dlr=2e-6)

We have: 180 labels to be augmented


100%|██████████| 180/180 [3:46:35<00:00, 75.53s/it]


In [ ]:
insuf_ds_list = list(insuf_ds_dict.values())

In [ ]:
gan_aug_train = torch.utils.data.ConcatDataset([suf_dataset]+augmented_ds_lists+insuf_ds_list)
gan_aug_dl = DataLoader(gan_aug_train, shuffle=True, batch_size=20)

In [ ]:
classifier = EmbeddingClassifier(prd.num_class, bert)
classifier = model_runner.train_classifier(gan_aug_dl, test_dl, classifier, lr=2e-5, n_epochs=3)
predictions, labels = model_runner.predict(test_dl, classifier)
acc, f1, precision, recall, label_f1 = evaluate(labels, predictions)

100%|██████████| 7903/7903 [54:14<00:00,  2.43it/s]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Total ACC: 0.005
F1: 0.000
Precision: 0.000
Recall: 0.005
loss 1.9808055325469593 0 acc:  0.005050505050505051 f1:  5.077585506539931e-05 precision 2.5516192575808606e-05 recall 0.005050505050505051


100%|██████████| 7903/7903 [54:17<00:00,  2.43it/s]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Total ACC: 0.005
F1: 0.000
Precision: 0.000
Recall: 0.005
loss 1.3431780358676164 1 acc:  0.005050505050505051 f1:  5.0767348472156645e-05 precision 2.5511896197196752e-05 recall 0.005050505050505051


 14%|█▍        | 1143/7903 [07:50<44:55,  2.51it/s]

In [ ]:
classifier = model_runner.train_classifier(original_dl, test_dl, classifier, lr=2e-5, n_epochs=3)

In [ ]:
gan_aug_train = torch.utils.data.ConcatDataset([original_train_dataset]+augmented_ds_lists)
gan_aug_dl = DataLoader(gan_aug_train, shuffle=True, batch_size=40)

In [ ]:
classifier = EmbeddingClassifier(prd.num_class, bert)
classifier = model_runner.train_classifier(gan_aug_dl, test_dl, classifier, lr=2e-5, n_epochs=3)
predictions, labels = model_runner.predict(test_dl, classifier)
acc, f1, precision, recall, label_f1 = evaluate(labels, predictions)